In [1]:
import pandas as pd
from geopy.distance import geodesic
from geographiclib.geodesic import Geodesic
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv("./hackathon_data_main.csv", delimiter=";", encoding="utf-8")

df['Дата вылета'] = pd.to_datetime(df['Дата вылета'])
df['Время вылета'] = pd.to_datetime(df['Время вылета'], format='%H:%M').dt.time
df['Время прилета'] = pd.to_datetime(df['Время прилета'], format='%H:%M').dt.time
df['Номер рейса'] = df['Номер рейса'].astype(str)
df['Доход пасс'] = pd.to_numeric(df['Доход пасс'].str.replace(',', '.'))
df['LF Кабина'] = pd.to_numeric(df['LF Кабина'].str.replace(',', '.'))

df

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,Код кабины,Доход пасс,Пассажиры
0,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,28,0.6786,19,359,C,10048.02,19
1,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,24,0.2500,6,359,W,2013.02,6
2,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,264,0.4129,109,359,Y,29917.88,109
3,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,28,0.5357,15,359,C,6306.29,15
4,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,24,0.2917,7,359,W,3834.73,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,2024-12-29,6568,AER,LED,08:00:00,12:25:00,120,0.6417,77,319,Y,6504.72,77
19265,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,8,0.1250,1,319,C,29.73,1
19266,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,120,0.6333,76,319,Y,10016.20,77
19267,2024-12-29,6792,SVX,KJA,18:15:00,23:20:00,5,0.4000,2,SU9,W,162.20,2


In [3]:
cdf = df.copy()

In [4]:
cdf['Год'] = cdf['Дата вылета'].dt.year
cdf['Месяц'] = cdf['Дата вылета'].dt.month
cdf['День'] = cdf['Дата вылета'].dt.day
cdf['День недели'] = cdf['Дата вылета'].dt.dayofweek

In [5]:
# https://www.kaggle.com/datasets/samvelkoch/global-airports-iata-icao-timezone-geo
adf = pd.read_csv("./airports.csv", delimiter=",", encoding="utf-8")

ADF_CODE = "IATA"
ADF_LATITUDE = "GeoPointLat"
ADF_LONGITUDE = "GeoPointLong"
ADF_CITY = "City_Name"
ADF_COUNTRY = "Country_Name"
ADF_COUNTRY_CODE = "Country_CodeA2"

cdf = cdf.merge(
    adf[[ADF_CODE, ADF_LATITUDE, ADF_LONGITUDE, ADF_CITY, ADF_COUNTRY, ADF_COUNTRY_CODE]],
    left_on='Аэропорт вылета',
    right_on=ADF_CODE,
    how='left'
)

cdf = cdf.rename(columns={
    ADF_LATITUDE: 'Широта аэропорта вылета',
    ADF_LONGITUDE: 'Долгота аэропорта вылета',
    ADF_CITY: 'Город аэропорта вылета',
    ADF_COUNTRY: 'Страна аэропорта вылета',
    ADF_COUNTRY_CODE: 'Код страны аэропорта вылета'
}).drop(ADF_CODE, axis=1)

cdf = cdf.merge(
    adf[[ADF_CODE, ADF_LATITUDE, ADF_LONGITUDE, ADF_CITY, ADF_COUNTRY, ADF_COUNTRY_CODE]],
    left_on='Аэропорт прилета',
    right_on=ADF_CODE,
    how='left'
)

cdf = cdf.rename(columns={
    ADF_LATITUDE: 'Широта аэропорта прилета',
    ADF_LONGITUDE: 'Долгота аэропорта прилета',
    ADF_CITY: 'Город аэропорта прилета',
    ADF_COUNTRY: 'Страна аэропорта прилета',
    ADF_COUNTRY_CODE: 'Код страны аэропорта прилета'
}).drop(ADF_CODE, axis=1)

cdf['Расстояние между аэропортами'] = cdf.apply(lambda row: geodesic(
    (row["Широта аэропорта вылета"], row["Долгота аэропорта вылета"]),
    (row["Широта аэропорта прилета"], row["Долгота аэропорта прилета"])
).kilometers, axis=1)

cdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,...,Долгота аэропорта вылета,Город аэропорта вылета,Страна аэропорта вылета,Код страны аэропорта вылета,Широта аэропорта прилета,Долгота аэропорта прилета,Город аэропорта прилета,Страна аэропорта прилета,Код страны аэропорта прилета,Расстояние между аэропортами
0,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,28,0.6786,19,359,...,37.416252,Moscow,Russian Federation,RU,18.302897,109.412272,Sanya,China,CN,7220.914732
1,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,24,0.2500,6,359,...,37.416252,Moscow,Russian Federation,RU,18.302897,109.412272,Sanya,China,CN,7220.914732
2,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,264,0.4129,109,359,...,37.416252,Moscow,Russian Federation,RU,18.302897,109.412272,Sanya,China,CN,7220.914732
3,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,28,0.5357,15,359,...,109.412272,Sanya,China,CN,55.964982,37.416252,Moscow,Russian Federation,RU,7220.914732
4,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,24,0.2917,7,359,...,109.412272,Sanya,China,CN,55.964982,37.416252,Moscow,Russian Federation,RU,7220.914732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,2024-12-29,6568,AER,LED,08:00:00,12:25:00,120,0.6417,77,319,...,39.956589,Sochi,Russian Federation,RU,59.800292,30.262503,Saint Petersburg,Russian Federation,RU,1934.265984
19265,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,8,0.1250,1,319,...,92.475000,Krasnoyarsk,Russian Federation,RU,59.800292,30.262503,Saint Petersburg,Russian Federation,RU,3564.618155
19266,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,120,0.6333,76,319,...,92.475000,Krasnoyarsk,Russian Federation,RU,59.800292,30.262503,Saint Petersburg,Russian Federation,RU,3564.618155
19267,2024-12-29,6792,SVX,KJA,18:15:00,23:20:00,5,0.4000,2,SU9,...,60.802680,Yekaterinburg,Russian Federation,RU,56.180000,92.475000,Krasnoyarsk,Russian Federation,RU,1936.039938


In [6]:
cdf['Международный рейс'] = cdf['Страна аэропорта вылета'] != cdf['Страна аэропорта прилета']
cdf['Вектор движения'] = cdf.apply(lambda row: Geodesic.WGS84.Inverse(
    row['Широта аэропорта вылета'], row['Долгота аэропорта вылета'],
    row['Широта аэропорта прилета'], row['Долгота аэропорта прилета'],
)['azi1'], axis=1)

cdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,...,Страна аэропорта вылета,Код страны аэропорта вылета,Широта аэропорта прилета,Долгота аэропорта прилета,Город аэропорта прилета,Страна аэропорта прилета,Код страны аэропорта прилета,Расстояние между аэропортами,Международный рейс,Вектор движения
0,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,28,0.6786,19,359,...,Russian Federation,RU,18.302897,109.412272,Sanya,China,CN,7220.914732,True,94.161933
1,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,24,0.2500,6,359,...,Russian Federation,RU,18.302897,109.412272,Sanya,China,CN,7220.914732,True,94.161933
2,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,264,0.4129,109,359,...,Russian Federation,RU,18.302897,109.412272,Sanya,China,CN,7220.914732,True,94.161933
3,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,28,0.5357,15,359,...,China,CN,55.964982,37.416252,Moscow,Russian Federation,RU,7220.914732,True,-36.095347
4,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,24,0.2917,7,359,...,China,CN,55.964982,37.416252,Moscow,Russian Federation,RU,7220.914732,True,-36.095347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,2024-12-29,6568,AER,LED,08:00:00,12:25:00,120,0.6417,77,319,...,Russian Federation,RU,59.800292,30.262503,Saint Petersburg,Russian Federation,RU,1934.265984,False,-16.518822
19265,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,8,0.1250,1,319,...,Russian Federation,RU,59.800292,30.262503,Saint Petersburg,Russian Federation,RU,3564.618155,False,-57.261267
19266,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,120,0.6333,76,319,...,Russian Federation,RU,59.800292,30.262503,Saint Petersburg,Russian Federation,RU,3564.618155,False,-57.261267
19267,2024-12-29,6792,SVX,KJA,18:15:00,23:20:00,5,0.4000,2,SU9,...,Russian Federation,RU,56.180000,92.475000,Krasnoyarsk,Russian Federation,RU,1936.039938,False,78.494856


In [7]:
gdf = pd.read_csv("./geonames-all-cities-with-a-population-1000.csv", delimiter=";", encoding="utf-8")
gdf['Alternate Names'] = gdf['Alternate Names'].astype(str)
gdf['Alternate Names'] = gdf['Alternate Names'].str.split(',')
gdf['all_names'] = gdf.apply(lambda row: list(set(
    [row['Name']] + 
    [row['ASCII Name']] + 
    ([] if row['Alternate Names'] == ['nan'] else row['Alternate Names'])
)), axis=1)
gdf = gdf.explode("all_names")
gdf['all_names'] = gdf['all_names'].str.lower()

# kaliningrad-korolev issue
gdf = gdf.sort_values('Population', ascending=False)
gdf = gdf.drop_duplicates(subset=['all_names', 'Country Code'], keep='first')

gdf

,Geoname ID,Name,ASCII Name,Alternate Names,Feature Class,Feature Code,Country Code,Country name EN,Country Code 2,Admin1 Code,...,Admin3 Code,Admin4 Code,Population,Elevation,DIgital Elevation Model,Timezone,Modification date,LABEL EN,Coordinates,all_names
44541,1796236,Shanghai,Shanghai,"[SHA, San'nkae, Sanchajus, Sangaj, Sangay, San...",P,PPLA,CN,China,NaN,23,...,NaN,NaN,22315474,NaN,12,Asia/Shanghai,2023-10-15,China,"31.22222, 121.45806",šanghaj
44541,1796236,Shanghai,Shanghai,"[SHA, San'nkae, Sanchajus, Sangaj, Sangay, San...",P,PPLA,CN,China,NaN,23,...,NaN,NaN,22315474,NaN,12,Asia/Shanghai,2023-10-15,China,"31.22222, 121.45806",sha
44541,1796236,Shanghai,Shanghai,"[SHA, San'nkae, Sanchajus, Sangaj, Sangay, San...",P,PPLA,CN,China,NaN,23,...,NaN,NaN,22315474,NaN,12,Asia/Shanghai,2023-10-15,China,"31.22222, 121.45806",შანჰაი
44541,1796236,Shanghai,Shanghai,"[SHA, San'nkae, Sanchajus, Sangaj, Sangay, San...",P,PPLA,CN,China,NaN,23,...,NaN,NaN,22315474,NaN,12,Asia/Shanghai,2023-10-15,China,"31.22222, 121.45806",shang hai
44541,1796236,Shanghai,Shanghai,"[SHA, San'nkae, Sanchajus, Sangaj, Sangay, San...",P,PPLA,CN,China,NaN,23,...,NaN,NaN,22315474,NaN,12,Asia/Shanghai,2023-10-15,China,"31.22222, 121.45806",shang hai shi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147036,130451,Jam,Jam,"[Jam, Jam-e Jam, Jamm, Jām-e Jam, Velayat, Vel...",P,PPLA2,IR,"Iran, Islamic Rep. of",NaN,22,...,NaN,NaN,0,NaN,637,Asia/Tehran,2014-09-04,"Iran, Islamic Rep. of","27.82817, 52.32536",jam-e jam
147036,130451,Jam,Jam,"[Jam, Jam-e Jam, Jamm, Jām-e Jam, Velayat, Vel...",P,PPLA2,IR,"Iran, Islamic Rep. of",NaN,22,...,NaN,NaN,0,NaN,637,Asia/Tehran,2014-09-04,"Iran, Islamic Rep. of","27.82817, 52.32536",jm
147036,130451,Jam,Jam,"[Jam, Jam-e Jam, Jamm, Jām-e Jam, Velayat, Vel...",P,PPLA2,IR,"Iran, Islamic Rep. of",NaN,22,...,NaN,NaN,0,NaN,637,Asia/Tehran,2014-09-04,"Iran, Islamic Rep. of","27.82817, 52.32536",jami jam
147036,130451,Jam,Jam,"[Jam, Jam-e Jam, Jamm, Jām-e Jam, Velayat, Vel...",P,PPLA2,IR,"Iran, Islamic Rep. of",NaN,22,...,NaN,NaN,0,NaN,637,Asia/Tehran,2014-09-04,"Iran, Islamic Rep. of","27.82817, 52.32536",velāyat


In [8]:
rdf = cdf.copy()

rdf['city_dep'] = rdf["Город аэропорта вылета"].str.lower()
rdf['city_arr'] = rdf["Город аэропорта прилета"].str.lower()

rdf = rdf.merge(
    gdf[["all_names", "Population", "Timezone", "Country Code"]],
    left_on=['city_dep', "Код страны аэропорта вылета"],
    right_on=["all_names", "Country Code"],
    how='left'
)

rdf = rdf.rename(columns={
    "Population": "Популяция аэропорта вылета",
    "Timezone": "Часовой пояс аэропорта вылета"
}).drop(["all_names", "city_dep", "Country Code"], axis=1)

rdf = rdf.merge(
    gdf[["all_names", "Population", "Timezone", "Country Code"]],
    left_on=['city_arr', "Код страны аэропорта прилета"],
    right_on=["all_names", "Country Code"],
    how='left'
)

rdf = rdf.rename(columns={
    "Population": "Популяция аэропорта прилета",
    "Timezone": "Часовой пояс аэропорта прилета"
}).drop(["all_names", "city_arr", "Country Code"], axis=1)

rdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,...,Город аэропорта прилета,Страна аэропорта прилета,Код страны аэропорта прилета,Расстояние между аэропортами,Международный рейс,Вектор движения,Популяция аэропорта вылета,Часовой пояс аэропорта вылета,Популяция аэропорта прилета,Часовой пояс аэропорта прилета
0,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,28,0.6786,19,359,...,Sanya,China,CN,7220.914732,True,94.161933,10381222,Europe/Moscow,1031396,Asia/Shanghai
1,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,24,0.2500,6,359,...,Sanya,China,CN,7220.914732,True,94.161933,10381222,Europe/Moscow,1031396,Asia/Shanghai
2,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,264,0.4129,109,359,...,Sanya,China,CN,7220.914732,True,94.161933,10381222,Europe/Moscow,1031396,Asia/Shanghai
3,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,28,0.5357,15,359,...,Moscow,Russian Federation,RU,7220.914732,True,-36.095347,1031396,Asia/Shanghai,10381222,Europe/Moscow
4,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,24,0.2917,7,359,...,Moscow,Russian Federation,RU,7220.914732,True,-36.095347,1031396,Asia/Shanghai,10381222,Europe/Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,2024-12-29,6568,AER,LED,08:00:00,12:25:00,120,0.6417,77,319,...,Saint Petersburg,Russian Federation,RU,1934.265984,False,-16.518822,327608,Europe/Moscow,5351935,Europe/Moscow
19265,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,8,0.1250,1,319,...,Saint Petersburg,Russian Federation,RU,3564.618155,False,-57.261267,1090811,Asia/Krasnoyarsk,5351935,Europe/Moscow
19266,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,120,0.6333,76,319,...,Saint Petersburg,Russian Federation,RU,3564.618155,False,-57.261267,1090811,Asia/Krasnoyarsk,5351935,Europe/Moscow
19267,2024-12-29,6792,SVX,KJA,18:15:00,23:20:00,5,0.4000,2,SU9,...,Krasnoyarsk,Russian Federation,RU,1936.039938,False,78.494856,1495066,Asia/Yekaterinburg,1090811,Asia/Krasnoyarsk


In [9]:
def calculate_row_timedelta(row):
    date_dep = row["Дата вылета"]
    time_dep = row["Время вылета"]
    time_arr = row["Время прилета"]
    
    datetime_dep = datetime.combine(date_dep, time_dep)
    datetime_arr = datetime.combine(date_dep, time_arr)
    
    if datetime_arr < datetime_dep:
        datetime_arr += timedelta(days=1)
    
    return (datetime_arr - datetime_dep).total_seconds() / 60

rdf["Время в пути"] = rdf.apply(calculate_row_timedelta, axis=1)
rdf["Время вылета (минуты)"] = rdf["Время вылета"].apply(lambda x: x.hour*60 + x.minute)
rdf["Время прилета (минуты)"] = rdf["Время прилета"].apply(lambda x: x.hour*60 + x.minute)
rdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,...,Расстояние между аэропортами,Международный рейс,Вектор движения,Популяция аэропорта вылета,Часовой пояс аэропорта вылета,Популяция аэропорта прилета,Часовой пояс аэропорта прилета,Время в пути,Время вылета (минуты),Время прилета (минуты)
0,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,28,0.6786,19,359,...,7220.914732,True,94.161933,10381222,Europe/Moscow,1031396,Asia/Shanghai,880.0,635,75
1,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,24,0.2500,6,359,...,7220.914732,True,94.161933,10381222,Europe/Moscow,1031396,Asia/Shanghai,880.0,635,75
2,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,264,0.4129,109,359,...,7220.914732,True,94.161933,10381222,Europe/Moscow,1031396,Asia/Shanghai,880.0,635,75
3,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,28,0.5357,15,359,...,7220.914732,True,-36.095347,1031396,Asia/Shanghai,10381222,Europe/Moscow,320.0,290,610
4,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,24,0.2917,7,359,...,7220.914732,True,-36.095347,1031396,Asia/Shanghai,10381222,Europe/Moscow,320.0,290,610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,2024-12-29,6568,AER,LED,08:00:00,12:25:00,120,0.6417,77,319,...,1934.265984,False,-16.518822,327608,Europe/Moscow,5351935,Europe/Moscow,265.0,480,745
19265,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,8,0.1250,1,319,...,3564.618155,False,-57.261267,1090811,Asia/Krasnoyarsk,5351935,Europe/Moscow,70.0,1195,1265
19266,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,120,0.6333,76,319,...,3564.618155,False,-57.261267,1090811,Asia/Krasnoyarsk,5351935,Europe/Moscow,70.0,1195,1265
19267,2024-12-29,6792,SVX,KJA,18:15:00,23:20:00,5,0.4000,2,SU9,...,1936.039938,False,78.494856,1495066,Asia/Yekaterinburg,1090811,Asia/Krasnoyarsk,305.0,1095,1400


In [10]:
from zoneinfo import ZoneInfo

def tz_difference(row) -> int:
    dt = datetime.now()
    
    # Локализуем дату в каждом часовом поясе
    dt1 = dt.astimezone(ZoneInfo(row['Часовой пояс аэропорта вылета']))
    dt2 = dt.astimezone(ZoneInfo(row['Часовой пояс аэропорта прилета']))
    
    # Разница в секундах → часы
    diff_hours = (dt1.utcoffset() - dt2.utcoffset()).total_seconds() / 3600
    return diff_hours

rdf["Разница часовых поясов"] = rdf.apply(tz_difference, axis=1)
rdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,...,Международный рейс,Вектор движения,Популяция аэропорта вылета,Часовой пояс аэропорта вылета,Популяция аэропорта прилета,Часовой пояс аэропорта прилета,Время в пути,Время вылета (минуты),Время прилета (минуты),Разница часовых поясов
0,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,28,0.6786,19,359,...,True,94.161933,10381222,Europe/Moscow,1031396,Asia/Shanghai,880.0,635,75,-5.0
1,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,24,0.2500,6,359,...,True,94.161933,10381222,Europe/Moscow,1031396,Asia/Shanghai,880.0,635,75,-5.0
2,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,264,0.4129,109,359,...,True,94.161933,10381222,Europe/Moscow,1031396,Asia/Shanghai,880.0,635,75,-5.0
3,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,28,0.5357,15,359,...,True,-36.095347,1031396,Asia/Shanghai,10381222,Europe/Moscow,320.0,290,610,5.0
4,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,24,0.2917,7,359,...,True,-36.095347,1031396,Asia/Shanghai,10381222,Europe/Moscow,320.0,290,610,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,2024-12-29,6568,AER,LED,08:00:00,12:25:00,120,0.6417,77,319,...,False,-16.518822,327608,Europe/Moscow,5351935,Europe/Moscow,265.0,480,745,0.0
19265,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,8,0.1250,1,319,...,False,-57.261267,1090811,Asia/Krasnoyarsk,5351935,Europe/Moscow,70.0,1195,1265,4.0
19266,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,120,0.6333,76,319,...,False,-57.261267,1090811,Asia/Krasnoyarsk,5351935,Europe/Moscow,70.0,1195,1265,4.0
19267,2024-12-29,6792,SVX,KJA,18:15:00,23:20:00,5,0.4000,2,SU9,...,False,78.494856,1495066,Asia/Yekaterinburg,1090811,Asia/Krasnoyarsk,305.0,1095,1400,-2.0


In [11]:
VALUE_COLS = ['Бронирования по кабинам', 'Доход пасс', 'Пассажиры']
GROUP_COLS = ['Номер рейса']

pdf = rdf.copy()

pdf['temp_week'] = pdf['Дата вылета'].dt.to_period('W').apply(lambda r: r.start_time)
pdf['temp_week2'] = (pdf['Дата вылета'].dt.isocalendar().week // 2).astype(int)
pdf['temp_month'] = pdf['Дата вылета'].dt.to_period('M').astype(str)

# pdf['temp_year'] = pdf['Дата вылета'].dt.year  # TODO: считать вместе с годом, но датасет кнш 10 месяцев

pdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,...,Часовой пояс аэропорта вылета,Популяция аэропорта прилета,Часовой пояс аэропорта прилета,Время в пути,Время вылета (минуты),Время прилета (минуты),Разница часовых поясов,temp_week,temp_week2,temp_month
0,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,28,0.6786,19,359,...,Europe/Moscow,1031396,Asia/Shanghai,880.0,635,75,-5.0,2025-04-14,8,2025-04
1,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,24,0.2500,6,359,...,Europe/Moscow,1031396,Asia/Shanghai,880.0,635,75,-5.0,2025-04-14,8,2025-04
2,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,264,0.4129,109,359,...,Europe/Moscow,1031396,Asia/Shanghai,880.0,635,75,-5.0,2025-04-14,8,2025-04
3,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,28,0.5357,15,359,...,Asia/Shanghai,10381222,Europe/Moscow,320.0,290,610,5.0,2025-04-14,8,2025-04
4,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,24,0.2917,7,359,...,Asia/Shanghai,10381222,Europe/Moscow,320.0,290,610,5.0,2025-04-14,8,2025-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,2024-12-29,6568,AER,LED,08:00:00,12:25:00,120,0.6417,77,319,...,Europe/Moscow,5351935,Europe/Moscow,265.0,480,745,0.0,2024-12-23,26,2024-12
19265,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,8,0.1250,1,319,...,Asia/Krasnoyarsk,5351935,Europe/Moscow,70.0,1195,1265,4.0,2024-12-23,26,2024-12
19266,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,120,0.6333,76,319,...,Asia/Krasnoyarsk,5351935,Europe/Moscow,70.0,1195,1265,4.0,2024-12-23,26,2024-12
19267,2024-12-29,6792,SVX,KJA,18:15:00,23:20:00,5,0.4000,2,SU9,...,Asia/Yekaterinburg,1090811,Asia/Krasnoyarsk,305.0,1095,1400,-2.0,2024-12-23,26,2024-12


In [12]:
daily = pdf.groupby(GROUP_COLS + ['Дата вылета'])[VALUE_COLS].sum().reset_index()
for col in VALUE_COLS:
    pdf = pdf.merge(daily[GROUP_COLS + ['Дата вылета', col]],
                    on=GROUP_COLS + ['Дата вылета'], how='left',
                    suffixes=('', f' (рейс/день)'))

weekly = pdf.groupby(GROUP_COLS + ['temp_week'])[VALUE_COLS].sum().reset_index()
for col in VALUE_COLS:
    pdf = pdf.merge(weekly[GROUP_COLS + ['temp_week', col]],
                    on=GROUP_COLS + ['temp_week'], how='left',
                    suffixes=('', f' (рейс/неделя)'))

biweekly = pdf.groupby(GROUP_COLS + ['temp_week2'])[VALUE_COLS].sum().reset_index()
for col in VALUE_COLS:
    pdf = pdf.merge(biweekly[GROUP_COLS + ['temp_week2', col]],
                    on=GROUP_COLS + ['temp_week2'], how='left',
                    suffixes=('', f' (рейс/2 недели)'))

monthly = pdf.groupby(GROUP_COLS + ['temp_month'])[VALUE_COLS].sum().reset_index()
for col in VALUE_COLS:
    pdf = pdf.merge(monthly[GROUP_COLS + ['temp_month', col]],
                    on=GROUP_COLS + ['temp_month'], how='left',
                    suffixes=('', f' (рейс/месяц)'))
    
pdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,...,Пассажиры (рейс/день),Бронирования по кабинам (рейс/неделя),Доход пасс (рейс/неделя),Пассажиры (рейс/неделя),Бронирования по кабинам (рейс/2 недели),Доход пасс (рейс/2 недели),Пассажиры (рейс/2 недели),Бронирования по кабинам (рейс/месяц),Доход пасс (рейс/месяц),Пассажиры (рейс/месяц)
0,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,28,0.6786,19,359,...,134,524,146304.98,526,1216,388196.71,1216,2693,851751.63,2700
1,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,24,0.2500,6,359,...,134,524,146304.98,526,1216,388196.71,1216,2693,851751.63,2700
2,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,264,0.4129,109,359,...,134,524,146304.98,526,1216,388196.71,1216,2693,851751.63,2700
3,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,28,0.5357,15,359,...,164,755,242504.70,764,1430,462319.79,1443,2989,970736.50,3007
4,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,24,0.2917,7,359,...,164,755,242504.70,764,1430,462319.79,1443,2989,970736.50,3007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,2024-12-29,6568,AER,LED,08:00:00,12:25:00,120,0.6417,77,319,...,80,465,46029.01,467,465,46029.01,467,1483,136066.64,1493
19265,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,8,0.1250,1,319,...,78,667,88268.31,669,667,88268.31,669,3161,328841.20,3195
19266,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,120,0.6333,76,319,...,78,667,88268.31,669,667,88268.31,669,3161,328841.20,3195
19267,2024-12-29,6792,SVX,KJA,18:15:00,23:20:00,5,0.4000,2,SU9,...,100,446,34071.65,449,446,34071.65,449,2013,142078.23,2028


In [13]:
daily_all = pdf.groupby(['Дата вылета'])[VALUE_COLS].sum().reset_index()
for col in VALUE_COLS:
    pdf = pdf.merge(daily_all[['Дата вылета', col]],
                    on='Дата вылета', how='left',
                    suffixes=('', f' (все рейсы/день)'))

weekly_all = pdf.groupby(['temp_week'])[VALUE_COLS].sum().reset_index()
for col in VALUE_COLS:
    pdf = pdf.merge(weekly_all[['temp_week', col]],
                    on='temp_week', how='left',
                    suffixes=('', f' (все рейсы/неделя)'))

biweekly_all = pdf.groupby(['temp_week2'])[VALUE_COLS].sum().reset_index()
for col in VALUE_COLS:
    pdf = pdf.merge(biweekly_all[['temp_week2', col]],
                    on=['temp_week2'], how='left',
                    suffixes=('', f' (все рейсы/2 недели)'))

monthly_all = pdf.groupby(['temp_month'])[VALUE_COLS].sum().reset_index()
for col in VALUE_COLS:
    pdf = pdf.merge(monthly_all[['temp_month', col]],
                    on='temp_month', how='left',
                    suffixes=('', f' (все рейсы/месяц)'))

pdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,...,Пассажиры (все рейсы/день),Бронирования по кабинам (все рейсы/неделя),Доход пасс (все рейсы/неделя),Пассажиры (все рейсы/неделя),Бронирования по кабинам (все рейсы/2 недели),Доход пасс (все рейсы/2 недели),Пассажиры (все рейсы/2 недели),Бронирования по кабинам (все рейсы/месяц),Доход пасс (все рейсы/месяц),Пассажиры (все рейсы/месяц)
0,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,28,0.6786,19,359,...,4673,28435,4639129.30,28693,60020,9819478.47,60543,132628,22625138.28,133834
1,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,24,0.2500,6,359,...,4673,28435,4639129.30,28693,60020,9819478.47,60543,132628,22625138.28,133834
2,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,264,0.4129,109,359,...,4673,28435,4639129.30,28693,60020,9819478.47,60543,132628,22625138.28,133834
3,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,28,0.5357,15,359,...,4673,28435,4639129.30,28693,60020,9819478.47,60543,132628,22625138.28,133834
4,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,24,0.2917,7,359,...,4673,28435,4639129.30,28693,60020,9819478.47,60543,132628,22625138.28,133834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,2024-12-29,6568,AER,LED,08:00:00,12:25:00,120,0.6417,77,319,...,4380,29724,5124828.28,30023,29724,5124828.28,30023,134998,21277306.46,136408
19265,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,8,0.1250,1,319,...,4380,29724,5124828.28,30023,29724,5124828.28,30023,134998,21277306.46,136408
19266,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,120,0.6333,76,319,...,4380,29724,5124828.28,30023,29724,5124828.28,30023,134998,21277306.46,136408
19267,2024-12-29,6792,SVX,KJA,18:15:00,23:20:00,5,0.4000,2,SU9,...,4380,29724,5124828.28,30023,29724,5124828.28,30023,134998,21277306.46,136408


In [14]:
pdf = pdf.drop(["temp_week", "temp_week2", "temp_month"], axis=1)
pdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,...,Пассажиры (все рейсы/день),Бронирования по кабинам (все рейсы/неделя),Доход пасс (все рейсы/неделя),Пассажиры (все рейсы/неделя),Бронирования по кабинам (все рейсы/2 недели),Доход пасс (все рейсы/2 недели),Пассажиры (все рейсы/2 недели),Бронирования по кабинам (все рейсы/месяц),Доход пасс (все рейсы/месяц),Пассажиры (все рейсы/месяц)
0,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,28,0.6786,19,359,...,4673,28435,4639129.30,28693,60020,9819478.47,60543,132628,22625138.28,133834
1,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,24,0.2500,6,359,...,4673,28435,4639129.30,28693,60020,9819478.47,60543,132628,22625138.28,133834
2,2025-04-18,224,SVO,SYX,10:35:00,01:15:00,264,0.4129,109,359,...,4673,28435,4639129.30,28693,60020,9819478.47,60543,132628,22625138.28,133834
3,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,28,0.5357,15,359,...,4673,28435,4639129.30,28693,60020,9819478.47,60543,132628,22625138.28,133834
4,2025-04-18,225,SYX,SVO,04:50:00,10:10:00,24,0.2917,7,359,...,4673,28435,4639129.30,28693,60020,9819478.47,60543,132628,22625138.28,133834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,2024-12-29,6568,AER,LED,08:00:00,12:25:00,120,0.6417,77,319,...,4380,29724,5124828.28,30023,29724,5124828.28,30023,134998,21277306.46,136408
19265,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,8,0.1250,1,319,...,4380,29724,5124828.28,30023,29724,5124828.28,30023,134998,21277306.46,136408
19266,2024-12-29,6574,KJA,LED,19:55:00,21:05:00,120,0.6333,76,319,...,4380,29724,5124828.28,30023,29724,5124828.28,30023,134998,21277306.46,136408
19267,2024-12-29,6792,SVX,KJA,18:15:00,23:20:00,5,0.4000,2,SU9,...,4380,29724,5124828.28,30023,29724,5124828.28,30023,134998,21277306.46,136408


In [15]:
tdf = pdf.copy()

tdf["Кол-во вылетов рейса (день)"] = (
    tdf.groupby(["Номер рейса", tdf["Дата вылета"].dt.date])["Номер рейса"]
      .transform("count")
)

tdf["Кол-во вылетов рейса (неделя)"] = (
    tdf.groupby(["Номер рейса", tdf["Дата вылета"].dt.to_period("W")])["Номер рейса"]
      .transform("count")
)

tdf["Кол-во вылетов рейса (2 недели)"] = (
    tdf.groupby(["Номер рейса", (tdf["Дата вылета"].dt.to_period("W").astype(int) // 2)])["Номер рейса"]
      .transform("count")
)

tdf["Кол-во вылетов рейса (месяц)"] = (
    tdf.groupby(["Номер рейса", tdf["Дата вылета"].dt.to_period("M")])["Номер рейса"]
      .transform("count")
)

tdf["Кол-во вылетов всех рейсов (день)"] = (
    tdf.groupby(tdf["Дата вылета"].dt.date)["Номер рейса"]
      .transform("count")
) # их всегда одна штука о_О

# неделя
tdf["Кол-во вылетов всех рейсов (неделя)"] = (
    tdf.groupby(tdf["Дата вылета"].dt.to_period("W"))["Номер рейса"]
      .transform("count")
)

# 2 недели
tdf["Кол-во вылетов всех рейсов (2 недели)"] = (
    tdf.groupby(tdf["Дата вылета"].dt.to_period("W").astype(int) // 2)["Номер рейса"]
      .transform("count")
)

# месяц
tdf["Кол-во вылетов всех рейсов (месяц)"] = (
    tdf.groupby(tdf["Дата вылета"].dt.to_period("M"))["Номер рейса"]
      .transform("count")
)

In [16]:
trdf = tdf.copy()

trdf = trdf.sort_values("Дата вылета")

trdf['Окно вылетов всех рейсов (день)'] = trdf.rolling(window="1D", on="Дата вылета")['Номер рейса'].count().reset_index(drop=True)
trdf['Окно вылетов всех рейсов (неделя)'] = trdf.rolling(window="7D", on="Дата вылета")['Номер рейса'].count().reset_index(drop=True)
trdf['Окно вылетов всех рейсов (2 недели)'] = trdf.rolling(window="14D", on="Дата вылета")['Номер рейса'].count().reset_index(drop=True)
trdf['Окно вылетов всех рейсов (месяц)'] = trdf.rolling(window="30D", on="Дата вылета")['Номер рейса'].count().reset_index(drop=True)

trdf['Окно вылетов рейса (день)'] = trdf.groupby('Номер рейса').rolling(window="1D", on="Дата вылета")['Дата вылета'].count().reset_index(drop=True)
trdf['Окно вылетов рейса (неделя)'] = trdf.groupby('Номер рейса').rolling(window="7D", on="Дата вылета")['Дата вылета'].count().reset_index(drop=True)
trdf['Окно вылетов рейса (2 недели)'] = trdf.groupby('Номер рейса').rolling(window="14D", on="Дата вылета")['Дата вылета'].count().reset_index(drop=True)
trdf['Окно вылетов рейса (месяц)'] = trdf.groupby('Номер рейса').rolling(window="30D", on="Дата вылета")['Дата вылета'].count().reset_index(drop=True)

trdf = trdf.reset_index(drop=True)

trdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,...,Кол-во вылетов всех рейсов (2 недели),Кол-во вылетов всех рейсов (месяц),Окно вылетов всех рейсов (день),Окно вылетов всех рейсов (неделя),Окно вылетов всех рейсов (2 недели),Окно вылетов всех рейсов (месяц),Окно вылетов рейса (день),Окно вылетов рейса (неделя),Окно вылетов рейса (2 недели),Окно вылетов рейса (месяц)
0,2024-08-01,1479,ABA,SVO,10:00:00,11:00:00,167,1.0299,172,32B,...,251,1979,39.0,403.0,842.0,1782.0,3.0,6.0,12.0,27.0
1,2024-08-01,427,SSH,SVO,16:30:00,22:30:00,28,0.1071,3,333,...,251,1979,8.0,372.0,811.0,1751.0,2.0,5.0,10.0,23.0
2,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,375,0.5467,205,77W,...,251,1979,7.0,371.0,810.0,1750.0,1.0,4.0,9.0,22.0
3,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,24,0.2500,6,77W,...,251,1979,6.0,370.0,809.0,1749.0,3.0,6.0,11.0,23.0
4,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,28,0.2500,7,77W,...,251,1979,5.0,369.0,808.0,1748.0,2.0,5.0,10.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,2025-06-02,1009,KGD,SVO,03:00:00,06:30:00,29,0.5517,16,32B,...,459,114,24.0,423.0,893.0,1843.0,2.0,14.0,28.0,60.0
19265,2025-06-02,1009,KGD,SVO,03:00:00,06:30:00,138,1.1014,152,32B,...,459,114,25.0,424.0,894.0,1844.0,1.0,13.0,27.0,59.0
19266,2025-06-02,1018,SVO,KGD,11:50:00,13:35:00,16,0.8750,14,32B,...,459,114,26.0,425.0,895.0,1845.0,2.0,14.0,28.0,60.0
19267,2025-06-02,1018,SVO,KGD,11:50:00,13:35:00,138,1.0290,142,32B,...,459,114,28.0,427.0,897.0,1847.0,2.0,14.0,28.0,60.0


In [17]:
df_expanded = trdf.copy()

QUANTILE_COLS = ["Доход пасс", "LF Кабина", "Бронирования по кабинам", "Пассажиры"]

for col_name in QUANTILE_COLS:
    df_expanded = df_expanded.merge(trdf.groupby(['Номер рейса'])[col_name].agg(**{
        f"Min {col_name} (Рейс)": lambda x: x.quantile(0.02),
        f"Max {col_name} (Рейс)": lambda x: x.quantile(0.98),
        f"Median {col_name} (Рейс)": 'median',
        f"IQR {col_name} (Рейс)": lambda x: x.quantile(0.75) - x.quantile(0.25)
    }).reset_index(), on=['Номер рейса'], how='left')
    
    df_expanded = df_expanded.merge(trdf.groupby(['Тип ВС', 'Код кабины', 'Емкость кабины'])[col_name].agg(**{
        f"Min {col_name} (ВС)": lambda x: x.quantile(0.02),
        f"Max {col_name} (ВС)": lambda x: x.quantile(0.98),
        f"Median {col_name} (ВС)": 'median',
        f"IQR {col_name} (ВС)": lambda x: x.quantile(0.75) - x.quantile(0.25)
    }).reset_index(), on=['Тип ВС', 'Код кабины', 'Емкость кабины'], how='left')

df_expanded

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,...,Median Бронирования по кабинам (ВС),IQR Бронирования по кабинам (ВС),Min Пассажиры (Рейс),Max Пассажиры (Рейс),Median Пассажиры (Рейс),IQR Пассажиры (Рейс),Min Пассажиры (ВС),Max Пассажиры (ВС),Median Пассажиры (ВС),IQR Пассажиры (ВС)
0,2024-08-01,1479,ABA,SVO,10:00:00,11:00:00,167,1.0299,172,32B,...,164.0,24.00,1.0,181.00,106.0,148.00,49.56,184.00,166.0,24.0
1,2024-08-01,427,SSH,SVO,16:30:00,22:30:00,28,0.1071,3,333,...,11.0,10.00,1.0,80.00,14.0,26.00,2.00,25.00,11.0,10.0
2,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,375,0.5467,205,77W,...,221.0,148.25,2.0,268.00,19.0,123.50,65.96,395.68,231.5,145.5
3,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,24,0.2500,6,77W,...,15.0,11.00,2.0,268.00,19.0,123.50,1.00,28.00,15.0,11.0
4,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,28,0.2500,7,77W,...,14.0,11.00,2.0,268.00,19.0,123.50,3.00,27.00,14.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,2025-06-02,1009,KGD,SVO,03:00:00,06:30:00,29,0.5517,16,32B,...,17.0,18.00,1.0,187.66,23.5,160.25,2.00,30.00,17.5,18.0
19265,2025-06-02,1009,KGD,SVO,03:00:00,06:30:00,138,1.1014,152,32B,...,147.0,13.00,1.0,187.66,23.5,160.25,131.00,177.24,147.0,13.0
19266,2025-06-02,1018,SVO,KGD,11:50:00,13:35:00,16,0.8750,14,32B,...,7.0,6.00,4.0,174.00,16.0,131.00,1.00,16.00,7.0,6.0
19267,2025-06-02,1018,SVO,KGD,11:50:00,13:35:00,138,1.0290,142,32B,...,147.0,13.00,4.0,174.00,16.0,131.00,131.00,177.24,147.0,13.0


In [18]:
df_expanded.to_csv("./prepared-dataset.csv", sep=";", encoding="utf-8", index=False)
df_expanded.to_pickle("./prepared-dataset.pkl")